In [2]:
import MDAnalysis as mda
from MDAnalysis.transformations.translate import translate
import numpy as np

/home/taylor/anaconda3/envs/dChem/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def rotmax(alpha, beta, gamma):
    matrix = np.array([[np.cos(beta)*np.cos(gamma),
                       -np.cos(gamma)*np.sin(alpha)*np.sin(beta) - np.cos(alpha)*np.sin(gamma),
                       -np.cos(alpha)*np.cos(gamma)*np.sin(beta) + np.sin(alpha)*np.sin(gamma)],
                      [np.cos(beta)*np.sin(gamma),
                       -np.sin(alpha)*np.sin(beta)*np.sin(gamma) + np.cos(alpha)*np.cos(gamma),
                       -np.cos(alpha)*np.sin(beta)*np.sin(gamma) - np.cos(gamma)*np.sin(alpha)],
                      [np.sin(beta),
                       np.cos(beta)*np.sin(alpha),
                       np.cos(alpha)*np.cos(beta)]])
    return matrix

In [3]:
alpha = 2.17401
beta = -0.63235
gamma = -2.06679
print(rotmax(alpha, beta, gamma))

[[-0.38388504 -0.73057018 -0.56471187]
 [-0.70943755 -0.15810233  0.68680566]
 [-0.59104199  0.66428222 -0.4576008 ]]


# Extract Top Score Transformation

In [4]:
def extractTopScore(filename):
    file = np.genfromtxt(filename, delimiter = "|", skip_header = 24, dtype = str)
    transform = file[0, 13]
    transform = transform.split(" ")
    transform = transform[1:]
    for j in range(len(transform)):
        transform[j] = float(transform[j])

    
    alpha = transform[0]
    beta = transform[1]
    gamma = transform[2]
    
    x = transform[3]
    y = transform[4]
    z = transform[5]
    
    return alpha, beta, gamma, x, y, z

In [17]:
protein = "d1exra_"
patchdock = f"{protein}_{protein}.txt"
print(extractTopScore(patchdock))

(-1.05964, -0.93373, 2.58407, -11.46599, 4.78695, -5.54074)


# Homodimer

In [5]:
def homodimer_pdb(protein, pdb, alpha, beta, gamma, x, y, z):
    
    rec_uni = mda.Universe(f"{pdb}")
    rec_atoms = rec_uni.select_atoms("all")
    rec_atoms.chainIDs = np.full([len(rec_atoms)], "A")
    
    lig_uni = mda.Universe(f"{pdb}")
    lig_atoms = lig_uni.select_atoms("all")
    lig_atoms.chainIDs = np.full([len(lig_atoms)], "B")
        
    lig_atoms.rotate(rotmax(alpha, beta, gamma))
    lig_atoms.translate([[x, y, z]])

    homodimer = mda.Merge(rec_atoms, lig_atoms)
    homodimer_atoms = homodimer.select_atoms("all")
    homodimer_atoms.write(f"{protein}_homodimer.pdb")
    return

In [15]:
for i in ["d2mspa_", "d3o6ca_", "d2pd1c_"]:
    protein = i
    pdb = f"../aligned/{protein}.pdb"
    patchdock = f"../homodimer_data/fine_patchdock/{protein}_{protein}.txt"
    alpha, beta, gamma, x, y, z = extractTopScore(patchdock)
    homodimer_pdb(protein, pdb, alpha, beta, gamma, x, y, z)

/home/taylor/anaconda3/envs/dChem/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: SE
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


# Heterodimer

In [ ]:
def heterodimer(receptor, ligand):